<a href="https://colab.research.google.com/github/b-richins92/Cancer_Prediction/blob/main/Grid_Search_Model_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import requests
from zipfile import ZipFile
from urllib.request import urlretrieve
import pandas as pd
import numpy as np
#import altair as alt
#from vega_datasets import data
%matplotlib inline
import seaborn as sns
sns.set()
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import statsmodels.api as sm
from sklearn.feature_selection import RFECV
from sklearn.ensemble import *
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.model_selection import *
from sklearn.metrics import *
from pycaret.classification import *
from imblearn.over_sampling import SMOTE
from sklearn.inspection import permutation_importance
import re
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import NearestNeighbors


In [ ]:
train_df_std = pd.read_csv(r'\Users\tegan\OneDrive\Desktop\Milestone2\train_df_std')
test_df_std = pd.read_csv(r'\Users\tegan\OneDrive\Desktop\Milestone2\test_df_std')
val_df_std = pd.read_csv(r'\Users\tegan\OneDrive\Desktop\Milestone2\val_df_std')

train_df_std.head()

,Gender,Age Group,Num of Bad Mental Health Days,Could Afford Doctor,Years Since Last Checkup,Exercise in Past 30 Days,Hours of Sleeping,Heart Attack,Heart Disease,Stroke,...,Ethnicity_Asian and Pacific Islander,Ethnicity_Black,Ethnicity_Hispanic,"Ethnicity_Multiracial, non-Hispanic",Ethnicity_White,Education_attended_college,Education_did_not_graduate_high_school,Education_graduated_college,Education_graduated_high_school,Cancer
0,1.0,1.508560,-0.528202,0.0,-0.448535,1.0,-0.684942,1.0,1.000000,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1.0,0.947918,-0.528202,0.0,-0.448535,1.0,-0.014824,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.667597,-0.528202,0.0,0.851674,1.0,0.655294,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
3,1.0,0.106955,-0.528202,0.0,-0.448535,0.0,-0.684942,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,0.0,1.228239,1.273841,0.0,-0.448535,0.0,1.995531,0.0,0.143527,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [ ]:
X_train, y_train = train_df_std.drop('Cancer', axis=1), train_df_std.Cancer
X_test, y_test = test_df_std.drop('Cancer', axis=1), test_df_std.Cancer
X_val, y_val = val_df_std.drop('Cancer', axis=1), val_df_std.Cancer

In [ ]:
smote = SMOTE(random_state=4)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# QDA

In [ ]:
qda = QuadraticDiscriminantAnalysis()
qda.fit(X_train_res, y_train_res)

QuadraticDiscriminantAnalysis()

In [ ]:
y_pred = qda.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

Accuracy: 0.5280538489338634
Precision: 0.1652955322111125
Recall: 0.7793912870499303
F1 Score: 0.27274625826662025


In [ ]:
# QDA Parameter Grid
param_grid = {
    'reg_param': [0.0, 0.01, 0.1, 0.5, 1.0],
    'store_covariance': [True, False],
    'tol': [1e-4, 1e-3, 1e-2, 1e-1, 1.0]
}

# Grid search
grid_search = GridSearchCV(estimator=qda, param_grid=param_grid, cv=5, scoring='recall')
grid_search.fit(X_train_res, y_train_res)

print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation score: ", grid_search.best_score_)

best_qda = grid_search.best_estimator_
print(best_qda)

Best parameters found:  {'reg_param': 0.1, 'store_covariance': True, 'tol': 0.0001}
Best cross-validation score:  0.8711877533623105
QuadraticDiscriminantAnalysis(reg_param=0.1, store_covariance=True)


In [ ]:
# Testing Grid Search QDA
grid_qda = QuadraticDiscriminantAnalysis(reg_param=0.,
                              store_covariance=True, tol=0.0001).fit(X_train_res, y_train_res)

y_pred = grid_qda.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print('Test Scores')
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

print(' ')
print('Validation Scores')
y_pred = grid_qda.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
precision = precision_score(y_val, y_pred)
recall = recall_score(y_val, y_pred)
f1 = f1_score(y_val, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')


# SVM

In [ ]:
# Baseline SVM Model
svm = SGDClassifier(random_state=42)
svm.fit(X_train_res, y_train_res)

y_pred = svm.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print('Test Scores')
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

Test Scores
Accuracy: 0.6352773762197326
Precision: 0.20578897237802943
Recall: 0.7736224388303163
F1 Score: 0.3250992685475444


In [ ]:
# SVM Parameter Grid
param_grid = {
    'loss': ['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron'],
    'penalty': ['l2', 'l1', 'elasticnet'],
    'alpha': [1e-6, 1e-4, 1e-3, 1e-2, 1e-1, 1],
    'l1_ratio': [0.0, 0.15, 0.5, 0.7, 1.0],
    'max_iter': [1000, 2000, 5000],
    'tol': [1e-4, 1e-3, 1e-2],
    'learning_rate': ['constant', 'optimal', 'invscaling', 'adaptive'],
    'eta0': [1e-4, 1e-3, 1e-2, 1e-1, 1],
    'power_t': [0.1, 0.25, 0.5, 0.75],
    'average': [False, True]
}

# Grid search
grid_search = GridSearchCV(estimator=svm, param_grid=param_grid, cv=5, scoring='recall')
grid_search.fit(X_train_res, y_train_res)

print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation score: ", grid_search.best_score_)

best_svm = grid_search.best_estimator_
print(best_svm)

In [ ]:
# Testing Grid Search QDA
grid_knn = INSERT BEST SVM MODEL HERE

y_pred = grid_svm.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print('Test Scores')
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

print(' ')
print('Validation Scores')
y_pred = grid_svm.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
precision = precision_score(y_val, y_pred)
recall = recall_score(y_val, y_pred)
f1 = f1_score(y_val, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')


# KNN

In [ ]:
# Baseline KNN Model
knn = NearestNeighibors(random_state=42)
knn.fit(X_train_res, y_train_res)

y_pred = knn.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print('Test Scores')
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

In [ ]:
# KNN Parameter Grid
param_grid = {
    'n_neighbors': [1, 3, 5, 7, 9, 11, 13, 15],
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'leaf_size': [10, 20, 30, 40, 50],
    'p': [1, 2],
    'metric': ['euclidean', 'manhattan', 'chebyshev', 'minkowski']
}

# Grid search
grid_search = GridSearchCV(estimator=knn, param_grid=param_grid, cv=5, scoring='recall')
grid_search.fit(X_train_res, y_train_res)

print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation score: ", grid_search.best_score_)

best_knn = grid_search.best_estimator_
print(best_knn)

In [ ]:
# Testing Grid Search KNN
grid_knn = INSERT BEST SVM MODEL HERE

y_pred = grid_knn.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print('Test Scores')
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

print(' ')
print('Validation Scores')
y_pred = grid_knn.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
precision = precision_score(y_val, y_pred)
recall = recall_score(y_val, y_pred)
f1 = f1_score(y_val, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')


# NB

In [ ]:
# Baseline NB Model
nb = GaussianNB()
nb.fit(X_train_res, y_train_res)

y_pred = nb.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print('Test Scores')
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

In [ ]:
# NB Parameter Grid
param_grid = {
    'alpha': [0.0, 0.1, 0.5, 1.0, 2.0, 5.0],
    'fit_prior': [True, False]
}

# Grid search
grid_search = GridSearchCV(estimator=knn, param_grid=param_grid, cv=5, scoring='recall')
grid_search.fit(X_train_res, y_train_res)

print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation score: ", grid_search.best_score_)

best_knn = grid_search.best_estimator_
print(best_knn)

In [ ]:
# Testing Grid Search QDA
grid_nb = INSERT BEST NB MODEL HERE

y_pred = grid_nb.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print('Test Scores')
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

print(' ')
print('Validation Scores')
y_pred = grid_nb.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
precision = precision_score(y_val, y_pred)
recall = recall_score(y_val, y_pred)
f1 = f1_score(y_val, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')
